In [38]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# Import data
df = pd.read_csv('nhldraft.csv')

df.shape

(12250, 16)

In [39]:
df.head(5)

id  year  overall_pick                 team               player  \
0   8813  1986           133    Vancouver Canucks         Jon Helgeson   
1     71  2022            71  Carolina Hurricanes  Alexander Perevalov   
2  11348  1974           102    Los Angeles Kings        Marty Mathews   
3   7600  1991           192  Pittsburgh Penguins          Jeff Lembke   
4    811  2019           147   New York Islanders        Reece Newkirk   

  nationality position   age  to_year                 amateur_team  \
0          US        C   NaN      NaN          Roseau HS (High-MN)   
1          RU       LW  18.0      NaN  Loko Yaroslavl (Russia Jr.)   
2          CA       LW   NaN      NaN     Flin Flon Bombers (WCHL)   
3          US        G   NaN      NaN         Omaha Lancers (USHL)   
4          CA        C  18.0      NaN   Portland Winterhawks (WHL)   

   games_played  goals  assists  points  plus_minus  penalties_minutes  
0           NaN    NaN      NaN     NaN         NaN                NaN  
1           NaN    NaN      NaN     NaN         NaN                NaN  
2           NaN    NaN      NaN     NaN         NaN                NaN  
3           NaN    NaN      NaN     NaN         NaN                NaN  
4           NaN    NaN      NaN     NaN         NaN                NaN

In [40]:
# Need to give a value to all NaN

df.fillna(0, inplace = True)
df.head(5)

id  year  overall_pick                 team               player  \
0   8813  1986           133    Vancouver Canucks         Jon Helgeson   
1     71  2022            71  Carolina Hurricanes  Alexander Perevalov   
2  11348  1974           102    Los Angeles Kings        Marty Mathews   
3   7600  1991           192  Pittsburgh Penguins          Jeff Lembke   
4    811  2019           147   New York Islanders        Reece Newkirk   

  nationality position   age  to_year                 amateur_team  \
0          US        C   0.0      0.0          Roseau HS (High-MN)   
1          RU       LW  18.0      0.0  Loko Yaroslavl (Russia Jr.)   
2          CA       LW   0.0      0.0     Flin Flon Bombers (WCHL)   
3          US        G   0.0      0.0         Omaha Lancers (USHL)   
4          CA        C  18.0      0.0   Portland Winterhawks (WHL)   

   games_played  goals  assists  points  plus_minus  penalties_minutes  
0           0.0    0.0      0.0     0.0         0.0                0.0  
1           0.0    0.0      0.0     0.0         0.0                0.0  
2           0.0    0.0      0.0     0.0         0.0                0.0  
3           0.0    0.0      0.0     0.0         0.0                0.0  
4           0.0    0.0      0.0     0.0         0.0                0.0

In [41]:
# Getting position counts of all - paying attention to goalies (G)
df['position'].value_counts()

position
D        3966
C        2688
LW       2080
RW       2021
G        1217
C/LW       74
C/RW       49
W          44
0          27
F          18
LW/C       18
LW/D        8
RW/C        8
D/LW        6
C/D         5
D/RW        4
RW/D        3
C/W         3
C RW        2
C; LW       2
D/C         2
C / R       2
Centr       1
L/RW        1
D/W         1
Name: count, dtype: int64

In [7]:
# Remove goalies from data and simplify position terms
# In the original dataset they had their own stat for games played

# List for indices
indices_to_drop = []

for index, row in df.iterrows():
    # Check if goalie
    if row['position'] == 'G':
        # Mark index to drop
        indices_to_drop.append(index)

# Remove marked indices
df.drop(indices_to_drop, inplace=True)

# Reset index of rows
df = df.reset_index(drop=True)
    
# TODO: If time - simplify duplicate positions
#if position == 'LW/C':
#print(position, i, df['position'][i+count])
#df.replace(['position'][i+count], 'C/LW', inplace=True)


In [8]:
# No goalies now
df['position'].value_counts()

position
D        3966
C        2688
LW       2080
RW       2021
C/LW       74
C/RW       49
W          44
0          27
LW/C       18
F          18
LW/D        8
RW/C        8
D/LW        6
C/D         5
D/RW        4
C/W         3
RW/D        3
C / R       2
C; LW       2
D/C         2
C RW        2
Centr       1
L/RW        1
D/W         1
Name: count, dtype: int64

In [9]:
# Decreased size, because of goalies removed
df.shape

(11033, 16)

In [10]:
df.head(5)

id  year  overall_pick                 team               player  \
0   8813  1986           133    Vancouver Canucks         Jon Helgeson   
1     71  2022            71  Carolina Hurricanes  Alexander Perevalov   
2  11348  1974           102    Los Angeles Kings        Marty Mathews   
3    811  2019           147   New York Islanders        Reece Newkirk   
4   1456  2016           141     New York Rangers    Timothy Gettinger   

  nationality position   age  to_year                 amateur_team  \
0          US        C   0.0      0.0          Roseau HS (High-MN)   
1          RU       LW  18.0      0.0  Loko Yaroslavl (Russia Jr.)   
2          CA       LW   0.0      0.0     Flin Flon Bombers (WCHL)   
3          CA        C  18.0      0.0   Portland Winterhawks (WHL)   
4          US       LW  18.0   2022.0         Soo Greyhounds (OHL)   

   games_played  goals  assists  points  plus_minus  penalties_minutes  
0           0.0    0.0      0.0     0.0         0.0                0.0  
1           0.0    0.0      0.0     0.0         0.0                0.0  
2           0.0    0.0      0.0     0.0         0.0                0.0  
3           0.0    0.0      0.0     0.0         0.0                0.0  
4          16.0    0.0      1.0     1.0        -1.0                0.0

In [11]:
# Label encode all values
le = LabelEncoder()

# Get all rows with strings
teams = [df['team'][i] for i in range(len(df))]
players = [df['player'][i] for i in range(len(df))]
nationalities = [df['nationality'][i] for i in range(len(df))]
positions = [df['position'][i] for i in range(len(df))]
amateur_teams = [df['amateur_team'][i] for i in range(len(df))]

# Label encode each row
teams = le.fit_transform(teams)
players = le.fit_transform(players)
nationalities = le.fit_transform(nationalities)
positions = le.fit_transform(positions)
amateur_teams = le.fit_transform(amateur_teams)

# Show new row
players

array([5233,  284, 6610, ..., 2234, 2075, 4705])

In [14]:
# Replace each column

df['team'] = teams
df['player'] = players
df['nationality'] = nationalities
df['position'] = positions
df['amateur_team'] = amateur_teams

df.head(5)

id  year  overall_pick  team  player  nationality  position   age  \
0   8813  1986           133    40    5233           42         1   0.0   
1     71  2022            71     9     284           33        17  18.0   
2  11348  1974           102    21    6610            8        17   0.0   
3    811  2019           147    27    8423            8         1  18.0   
4   1456  2016           141    28    9965           42        17  18.0   

   to_year  amateur_team  games_played  goals  assists  points  plus_minus  \
0      0.0          1062           0.0    0.0      0.0     0.0         0.0   
1      0.0           714           0.0    0.0      0.0     0.0         0.0   
2      0.0           358           0.0    0.0      0.0     0.0         0.0   
3      0.0          1007           0.0    0.0      0.0     0.0         0.0   
4   2022.0          1155          16.0    0.0      1.0     1.0        -1.0   

   penalties_minutes  
0                0.0  
1                0.0  
2                0.0  
3                0.0  
4                0.0

In [15]:
# Create X, removing games played

X = df.drop(columns=['games_played'])
X.head(10)

# Convert to numpy
X = X.to_numpy()

In [31]:
# Create y, with games played

df['games_played'].describe()

count    11033.000000
mean       137.152814
std        280.881682
min          0.000000
25%          0.000000
50%          0.000000
75%        101.000000
max       1779.000000
Name: games_played, dtype: float64

In [32]:
# Create y to be binary classification - 
# more or less games played than the mean
mean = 137
y = []
for new_y in range(len(df)):
    if df['games_played'][new_y] < mean:
        y.append(-1)
    else:
        y.append(1)

In [33]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 8826 2207


# Useful Functions

In [34]:
# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

def avg_grad(theta, X, y):
    grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    return np.mean(grads, axis=0)

# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

# L2 Clipping
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def gradient_sum(theta, X, y, b):
    gradients = [L2_clip(gradient(theta, x_i, y_i), b) for x_i, y_i in zip(X,y)]
        
    # sum query
    # L2 sensitivity is b (by clipping performed above)
    return np.sum(gradients, axis=0)
    
# Noisy gradient descent
# Satisfies (k*epsilon + epsilon, k*delta)-differential privacy
def noisy_gradient_descent(iterations, epsilon, delta):
    theta = np.zeros(X_train.shape[1])
    b = 3

    
    noisy_count = laplace_mech(X_train.shape[0], 1, epsilon)

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_vec(clipped_gradient_sum, b, epsilon, delta))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

# Baseline 
Use scikit-learn to train a logistic regression model on the training data loaded above.

In [42]:
def train_model():
    model = LogisticRegression().fit(X_train, y_train)
    return model

model = train_model()
#print('Model coefficients:', model.coef_[0])
print('Model accuracy:', np.sum(model.predict(X_test) == y_test)/X_test.shape[0])

Model accuracy: 0.9805165382872678


/Users/wyattchrisman/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Noisy Gradient Descent RDP

A variant of noisy gradient descent that uses Rényi differential privacy. With a total privacy cost of $(\alpha, \bar\epsilon)$-RDP.

In [36]:
def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def noisy_gradient_descent_RDP(iterations, alpha, epsilon_bar):
    theta = np.zeros(X_train.shape[1])
    b = 3
    
    split_epsilon = epsilon_bar / (iterations + 1)
    
    noisy_count = laplace_mech(X_train.shape[0], 1, split_epsilon)

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_RDP_vec(clipped_gradient_sum, b, alpha, split_epsilon))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta


theta = noisy_gradient_descent_RDP(500, 20, 0.1)
print('Final accuracy:', accuracy(theta))

/var/folders/yf/m10_kkm12wg1sd18gc0mp32c0000gn/T/ipykernel_24848/3086723542.py:11: RuntimeWarning: overflow encountered in exp
  return - (yi*xi) / (1+np.exp(exponent))


Final accuracy: 0.9311282283642954


## Noisy Gradient Descent zCDP
A variant of noisy gradient descent that uses zero-concentrated differential privacy. With a total privacy cost of $\rho$-zCDP.

In [37]:
def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]


def noisy_gradient_descent_zCDP(iterations, rho):
    theta = np.zeros(X_train.shape[1])
    b = 3
    
    split_rho = rho / (iterations + 1)
    
    noisy_count = gaussian_mech_zCDP_vec([X_train.shape[0]], 1, split_rho)

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_zCDP_vec(clipped_gradient_sum, b, split_rho))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

theta = noisy_gradient_descent_zCDP(500, 0.1)
print('Final accuracy:', accuracy(theta))

/var/folders/yf/m10_kkm12wg1sd18gc0mp32c0000gn/T/ipykernel_24848/3086723542.py:11: RuntimeWarning: overflow encountered in exp
  return - (yi*xi) / (1+np.exp(exponent))


Final accuracy: 0.8400543724512913
